# Import and install package

In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git
!conda install -c conda-forge keras-applications -y
!conda install -c conda-forge mtcnn -y
!conda install -c conda-forge keras -y
!conda install -c conda-forge tensorflow -y
!pip install keras-vggface
!pip install sklearn
!pip install keras_vggface

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from numpy import savetxt

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from keras_vggface.vggface import VGGFace
from tensorflow.keras.losses import sparse_categorical_crossentropy


# Datasets and Assign ID to dataset


In [ ]:
ATTRIBUTE_NAME = ['Bald','Bangs','Black_Hair','Blond_Hair','Brown_Hair','Bushy_Eyebrows','Eyeglasses','Male','Mouth_Slightly_Open','Mustache','No_Beard','Pale_Skin','Young'] #Attribute name from lfwa+ dataset
EXTRACT_NAME = 'ResNet50' #model to feature extraction

In [ ]:
ATTRIBUTE_NAME.sort() # sort for correct ordering 

In [ ]:
img_path1 = '/Users/bankthp/Downloads/GAN-DETECTION/crop_0to1/custom_testing_slide_{}/8'.format(ATTRIBUTE_NAME[0]) #path the generated image 0 to 1 
img_path2 = '/Users/bankthp/Downloads/GAN-DETECTION/crop_1to0/custom_testing_slide_{}/8'.format(ATTRIBUTE_NAME[0]) #path the generated image 1 to 0

pic0 = os.listdir(img_path1)
pic1 = os.listdir(img_path2)

pic0_size = round(len(pic0))
pic1_size = round(len(pic1))


pic_size = len(pic0+pic1)
all_pic = pic0+pic1

In [ ]:
all_pic.sort()

In [ ]:
all_pic 

In [ ]:
lfw_path = '/Users/bankthp/Downloads/GAN-DETECTION/lfw' #lfw path 
lfw_name = os.listdir('/Users/bankthp/Downloads/LFWA+/lfw') #lfw_name folder

In [ ]:
lfw_name.remove('.ipynb_checkpoints')
lfw_name.remove('.DS_Store')
lfw_name.sort()

In [ ]:
lfw_name

In [ ]:
pic_ids = range(len(all_pic))
pic_to_id = dict(zip(all_pic, pic_ids))

In [ ]:
pic_to_id

In [ ]:
ids = range(len(lfw_name))
name_to_id = dict(zip(lfw_name, ids))

In [ ]:
name_to_id

In [ ]:
import json

with open('userID.txt', 'w') as file:
     file.write(json.dumps(name_to_id))

# Feature Extraction

In [ ]:
extractResNet = ResNet50(weights='imagenet',include_top=False,pooling="avg")
model = extractResNet

img_path = img_path1+"/"+pic0[0]
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

In [ ]:
features.shape[1]

In [ ]:
for attr in ATTRIBUTE_NAME :
    print(attr)
    
    img_path1 = '/Users/bankthp/Downloads/GAN-DETECTION/crop_0to1/custom_testing_slide_{}/8'.format(attr)
    img_path2 = '/Users/bankthp/Downloads/GAN-DETECTION/crop_1to0/custom_testing_slide_{}/8'.format(attr)

    pic0 = os.listdir(img_path1)
    pic1 = os.listdir(img_path2)

    pic0_size = round(len(pic0))
    pic1_size = round(len(pic1))


    pic_size = len(pic0+pic1)
    
    all_pic = pic0+pic1
    
    lfw_path = '/Users/bankthp/Downloads/GAN-DETECTION/lfw'
    lfw_photo = os.listdir(lfw_path)
    
    
    
    size = len(all_pic)
    
    lfw_name = os.listdir('/Users/bankthp/Downloads/LFWA+/lfw')
    
    for name_list in [lfw_name, all_pic, lfw_photo]:
        if '.ipynb_checkpoints' in name_list:
            name_list.remove('.ipynb_checkpoints')
        if '.DS_Store' in name_list:
            name_list.remove('.DS_Store')
        
        
    lfw_name.sort()
    all_pic.sort()
    lfw_photo.sort()
    
    ids = range(len(lfw_name))
    name_to_id = dict(zip(lfw_name, ids))
    
    pic_ids = range(len(lfw_photo))
    pic_to_id = dict(zip(lfw_photo, pic_ids))
    
    
    count = 0

    feature = np.empty([size,features.shape[1]])

    train = pd.DataFrame({
        'user_id': np.zeros(size),
        'pic_id' : np.zeros(size),
        'pic_name' : np.empty((size),dtype=object),
        'attribute': np.empty((size), dtype=object),
        'real' : np.zeros(size)
    })


    for i in range(size) :
        
        #assgin dict id to train['user_id']
        name_no_ext = os.path.splitext(all_pic[i])[0]
        name_parts = name_no_ext.split('_')

        for j in range(len(name_parts)):
            sub_name = '_'.join(name_parts[:len(name_parts)-j])
            if sub_name in name_to_id:
                matched_name = sub_name
                break

        if matched_name:
            id_num = name_to_id[matched_name]
            train['user_id'][i] = id_num
        else:
            print("No match found")
            
        
        #assgin dict id to train['user_id']

        # check if the key "banana" exists in the dictionary
        if all_pic[i] in pic_to_id :
            train['pic_id'][i]= pic_to_id.get(all_pic[i])
        
            
            
        train['pic_name'][i] = all_pic[i]
        train['attribute'][i] = attr  
        train['real'][i] = 0
        feature[count,:] = model.predict(x)

        count = count + 1 
            
            
    train.to_csv('{}.csv'.format(attr),index=False)
    np.savetxt('feature_lfw.csv', feature, delimiter=',')

# LFW Feature Extraction

In [ ]:
lfw_path = '/Users/bankthp/Downloads/GAN-DETECTION/lfw'
lfw_photo = os.listdir(lfw_path)
    
size = len(all_pic)
    
lfw_name = os.listdir('/Users/bankthp/Downloads/LFWA+/lfw')
    
for name_list in [lfw_name, all_pic, lfw_photo]:
    if '.ipynb_checkpoints' in name_list:
        name_list.remove('.ipynb_checkpoints')
    if '.DS_Store' in name_list:
        name_list.remove('.DS_Store')
        
        
lfw_name.sort()
all_pic.sort()
lfw_photo.sort()
    
ids = range(len(lfw_name))
name_to_id = dict(zip(lfw_name, ids))
    
pic_ids = range(len(lfw_photo))
pic_to_id = dict(zip(lfw_photo, pic_ids))
    
    
count = 0
    
size = len(lfw_photo)

feature = np.empty([size,features.shape[1]])

train = pd.DataFrame({
    'user_id': np.zeros(size),
    'pic_id' : np.zeros(size),
    'pic_name' : np.empty((size),dtype=object),
    'attribute': np.empty((size), dtype=object),
    'real' : np.zeros(size)
})


for i in range(size) :
        
    #assgin dict id to train['user_id']
    name_no_ext = os.path.splitext(lfw_photo[i])[0]
    name_parts = name_no_ext.split('_')

    for j in range(len(name_parts)):
        sub_name = '_'.join(name_parts[:len(name_parts)-j])
        if sub_name in name_to_id:
            matched_name = sub_name
            break

    if matched_name:
        id_num = name_to_id[matched_name]
        train['user_id'][i] = id_num
    else:
        print("No match found")
            
        
    #assgin dict id to train['pic_id']
    if all_pic[i] in pic_to_id :
        train['pic_id'][i]= pic_to_id.get(lfw_photo[i])
        
            
            
    train['pic_name'][i] = lfw_photo[i]
    train['attribute'][i] = 'lfw'  
    train['real'][i] = 0
    feature[count,:] = model.predict(x)

    count = count + 1 
            
            
train.to_csv('lfw.csv',index=False)
np.savetxt('feature_lfw.csv', feature, delimiter=',')